Load data from files

In [16]:
import os
import json
import nltk

def getquestions():
    # data location
    question_dir = 'STEMquestions/data/'

    # list to store the extracted questions
    questions = []

    # Walk through the base directory and its subfolders
    for root, dirs, files in os.walk(question_dir):
        for file in files:
            if file.endswith(".json"):
                file_path = os.path.join(root, file)
                
                # Open and read the JSON file
                with open(file_path, 'r') as f:
                    data = json.load(f)
                    
                    # Extract the 'Original question' and add to the list
                    if "Original question" in data:
                        questions.append(data["Original question"])

    return questions

def getsentences(num_sentences):
    # Ensure you have the Punkt tokenizer models for sentence splitting (if not done already)
    # nltk.download('punkt')

    from nltk.tokenize import sent_tokenize

    # Define the base directory where your text files are located
    base_dir = 'STEMtext/'

    # Initialize a list to store the sentence chunks
    sentence_chunks = []

    # Number of sentences to extract
    sentences_per_chunk = 3  # Number of sentences per chunk (adjust as needed)

    # Loop through the text files in the directory
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                
                # Open and read the content of the text file
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                    
                    # Tokenize the content into individual sentences
                    sentences = sent_tokenize(content)
                    
                    # Group the sentences into chunks of sentences_per_chunk size
                    for i in range(0, len(sentences), sentences_per_chunk):
                        chunk = " ".join(sentences[i:i + sentences_per_chunk])
                        sentence_chunks.append(chunk)
                        
                        # Stop if we have collected enough sentence chunks
                        if len(sentence_chunks) >= num_sentences:
                            break

                # Stop outer loop if we have enough sentence chunks
                if len(sentence_chunks) >= num_sentences:
                    break

    return sentence_chunks
   

questions = getquestions()
sentences = getsentences(len(questions))
print(questions[0])
print(sentences[0])
print(len(questions), len(sentences))

Determine the kind of nonmarine sedimentary deposits that reflects arid environmental conditions
Integrality of the Chern character in small codimension

Abstract
We prove an integrality property of the Chern character with values in Chow groups. As a consequence, we obtain a construction of the p-1 first homological Steenrod operations on Chow groups modulo p and p-primary torsion, over an arbitrary field. We provide applications to the study of correspondences between algebraic varieties.
667 667


Encode sentences as vectors and create labels

In [26]:
from sentence_transformers import SentenceTransformer
import pandas as pd

def vector_encode(text):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    encoded = model.encode(text)
    return encoded.tolist()

questions_encoded = vector_encode(questions)
sentences_encoded = vector_encode(sentences)

question_labels = [1] * len(questions_encoded) # model outputs 1 ('true') for exam questions
sentence_labels = [0] * len(sentences_encoded)

# Create a DataFrame with columns for text, encodings, and labels
original_text = questions + sentences
encoded_data = questions_encoded + sentences_encoded
labels = question_labels + sentence_labels
df = pd.DataFrame({'original_text': original_text, 'data': encoded_data, 'labels': labels})

# Display the first few rows of the DataFrame (optional)
print(df.head())

c:\Users\jkamp\AppData\Local\pypoetry\Cache\virtualenvs\purdueprep-TEZ28JzX-py3.12\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


                                       original_text  \
0  Determine the kind of nonmarine sedimentary de...   
1  What is the major source of chlorine in seawat...   
2  Determine the use of actualism in geology to i...   
3  Determine the formation of an ion from uncharg...   
4  Determine the conditions that favor the preser...   

                                                data  labels  
0  [0.03205199912190437, 0.02383396029472351, 0.0...       1  
1  [-0.03738083317875862, -0.014038288034498692, ...       1  
2  [-0.08302020281553268, 0.08540865033864975, 0....       1  
3  [-0.02038067951798439, 0.0908573791384697, -0....       1  
4  [-0.10606085509061813, 0.10114321112632751, 0....       1  


Train K-nearest neighbors model

In [51]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(df['data'].tolist(), df['labels'], test_size=0.2, random_state=42)
# Initialize the K-Nearest Neighbors classifier
knn = KNeighborsClassifier(n_neighbors=1)

# Train the KNN model
knn.fit(X_train, y_train)

# Predict on the test set
y_pred = knn.predict(X_test)

# Evaluate the model with accuracy score
count_same = 0
for a, b in zip(y_pred, y_test):
    if a == b:
        count_same += 1
print(f"Model accuracy: {count_same / len(y_pred)}%")

Model accuracy: 0.9925093632958801%


test on new sentences

In [42]:
import numpy as np

test_sentences = getsentences(2000)
test_sentences = test_sentences[667:]
test_sentences_enc = vector_encode(test_sentences)

2000


c:\Users\jkamp\AppData\Local\pypoetry\Cache\virtualenvs\purdueprep-TEZ28JzX-py3.12\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [49]:
print(len(test_sentences_enc[0]))
preds = knn.predict(test_sentences_enc)
print(preds.tolist().count(1)/len(preds))

384
0.5918979744936234


In [52]:
print(np.linalg.norm(X_train[0] - test_sentences_enc[0]))  # Check similarity between a train and test vector


TypeError: unsupported operand type(s) for -: 'list' and 'list'